In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Dein erstet Quantenschaltkreis

Bau'n [Bell-Zustand](https://en.wikipedia.org/wiki/Bell_state) (zwee miteinanda verschränkte Qubits) und lass'n uff dreie Arten loofen:

1. **Ideale Simulation** — perfekte Erjebnisse, keent Konto nötich
2. **Simulation mit Rauschen** — simuliert 'n richtjet Jerät, keent Konto nötich
3. **Richtje Quantenhardware** — brauchst 'n [IBM Quantum-Konto](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Den Schaltkreis baun

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Option 1: Ideale Simulation (keent Konto nötich)
Nutzt `StatevectorSampler` — 'n lokalet Simulator mit perfekte, rauschfreie Erjebnisse.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Option 2: Simulation mit Rauschen (keent Konto nötich)
Nutzt `FakeManilaV2` — 'n lokalet Simulator, der 'n richtjet IBM-Quantenjerät nachmacht, ooch mit sein' Rauscheejenschaften. Der Schaltkreis muss erstma transpiliert (anjepaszt) wer'n, damit er zum Gate-Set un zur Qubit-Konnektivität vom Jerät passt.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Option 3: Richtje Quantenhardware
Brauchst 'n IBM Quantum-Konto. Kiek ma bei [Setting up your IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) für de Details.

Wenn de deine Zugangsdata noch nich in der Binder-Sitzung jespeichert hast, führ det erstma aus:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Achtung:** Jobs uff richtjer Hardware können 'n bisken dauern, je nachdem wie lang de Warteschlange is. Wenn die Zelle noch läuft, kannste den Job-Status und de Erjebnisse uff [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me) nachkieken.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Wat kommt als Nächstet?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — Schritt-für-Schritt-Anleitungen für Algorithmen, Fehlerverminderung, Transpilation und mehr
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — Strukturierte Lernpfade von Quantengrundlagen bis zu Utility-Scale-Computing
- **[Local testing mode](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — de meisten Notebooks ohne IBM Quantum-Konto ausführ'n